# 🚀 TIGAS Model Training on Cloud GPU

Notebook для обучения модели TIGAS на мощных GPU в **Kaggle** или **Google Colab**.

## Преимущества облачного обучения:
- 🔥 **Kaggle**: Tesla P100 (16GB) или T4 (16GB) бесплатно
- 🔥 **Colab**: Tesla T4 (15GB) или K80 (12GB) бесплатно
- ⚡ **Быстро**: ~6-12 часов вместо 5-7 дней на слабом GPU
- 🎯 **Batch size**: 16-32 вместо 2-4
- ❄️ **Нет проблем** с температурой и thermal throttling

## Инструкция:
1. Загрузите датасет TIGAS в Kaggle Dataset или Google Drive
2. Запустите все ячейки последовательно
3. Дождитесь завершения обучения
4. Скачайте обученную модель

## 1️⃣ Setup: Определение платформы и установка зависимостей

In [ ]:
# Определение платформы (Kaggle или Colab)
import os
import sys

# Detect platform
IS_KAGGLE = os.path.exists('/kaggle')
IS_COLAB = 'google.colab' in sys.modules

print("="*60)
if IS_KAGGLE:
    print("🎯 Платформа: KAGGLE")
    BASE_PATH = '/kaggle'
    DATASET_PATH = '/kaggle/input/tigas-dataset'  # Измените на ваш путь
elif IS_COLAB:
    print("🎯 Платформа: GOOGLE COLAB")
    BASE_PATH = '/content'
    DATASET_PATH = '/content/TIGAS'  # Измените на ваш путь
else:
    print("🎯 Платформа: LOCAL")
    BASE_PATH = '.'
    DATASET_PATH = 'C:/Dev/dataset/dataset/TIGAS_dataset/TIGAS'
print("="*60)

In [ ]:
# Проверка доступности GPU
import torch

print("\n🔍 Проверка GPU:")
print("="*60)
if torch.cuda.is_available():
    print(f"✅ GPU доступен: {torch.cuda.get_device_name(0)}")
    print(f"   Память: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"   CUDA версия: {torch.version.cuda}")
    device = 'cuda'
else:
    print("❌ GPU НЕ доступен! Используется CPU (медленно)")
    print("   Включите GPU в настройках: Runtime → Change runtime type → GPU")
    device = 'cpu'
print("="*60)

In [ ]:
# Установка дополнительных зависимостей (если нужно)
print("\n📦 Установка зависимостей...")
!pip install -q pandas pillow tqdm tensorboard
print("✅ Зависимости установлены")

## 2️⃣ Загрузка кода проекта TIGAS

### Вариант 1: Клонирование из GitHub (если репозиторий публичный)
```python
!git clone https://github.com/YOUR_USERNAME/TIGAS.git
%cd TIGAS
```

### Вариант 2: Upload файлов проекта
- В Colab: Files → Upload
- В Kaggle: Add Data → Upload Dataset

### Вариант 3: Копирование кода напрямую (выберите этот вариант)

In [ ]:
# Создание структуры проекта
import os
from pathlib import Path

# Создание директорий
os.makedirs('tigas/models', exist_ok=True)
os.makedirs('tigas/data', exist_ok=True)
os.makedirs('tigas/training', exist_ok=True)
os.makedirs('tigas/utils', exist_ok=True)
os.makedirs('checkpoints/logs', exist_ok=True)

print("✅ Структура проекта создана")
print("\n⚠️ ВАЖНО: Загрузите файлы проекта TIGAS:")
print("   - tigas/models/*.py")
print("   - tigas/data/*.py")
print("   - tigas/training/*.py")
print("   - tigas/utils/*.py")
print("\nИли клонируйте репозиторий:")
print("   !git clone YOUR_REPO_URL")

### 📁 Загрузка датасета TIGAS

**Для Kaggle:**
1. Добавьте TIGAS dataset в Kaggle Datasets
2. Add Data → Search → Select your dataset
3. Путь будет: `/kaggle/input/tigas-dataset/TIGAS`

**Для Colab:**
1. Загрузите датасет на Google Drive
2. Подключите Drive (запустите ячейку ниже)
3. Путь будет: `/content/drive/MyDrive/TIGAS`

In [ ]:
# Для Google Colab: подключение Google Drive
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive подключен")
    DATASET_PATH = '/content/drive/MyDrive/TIGAS'  # Измените на ваш путь
    print(f"📁 Путь к датасету: {DATASET_PATH}")

In [ ]:
# Проверка наличия датасета
import os
from pathlib import Path

print("\n🔍 Проверка датасета:")
print("="*60)

dataset_path = Path(DATASET_PATH)
if dataset_path.exists():
    print(f"✅ Датасет найден: {dataset_path}")
    
    # Проверка структуры
    train_csv = dataset_path / 'train' / 'annotations01.csv'
    val_csv = dataset_path / 'val' / 'annotations01.csv'
    test_csv = dataset_path / 'test' / 'annotations01.csv'
    
    if train_csv.exists():
        import pandas as pd
        train_count = len(pd.read_csv(train_csv))
        val_count = len(pd.read_csv(val_csv)) if val_csv.exists() else 0
        test_count = len(pd.read_csv(test_csv)) if test_csv.exists() else 0
        
        print(f"   📊 Train: {train_count:,} изображений")
        print(f"   📊 Val: {val_count:,} изображений")
        print(f"   📊 Test: {test_count:,} изображений")
        print(f"   📊 Всего: {train_count + val_count + test_count:,} изображений")
    else:
        print("   ⚠️ Файлы annotations01.csv не найдены!")
else:
    print(f"❌ Датасет НЕ найден по пути: {dataset_path}")
    print("   Измените DATASET_PATH на правильный путь к вашему датасету")
print("="*60)

## 3️⃣ Конфигурация обучения

Настройка параметров обучения под мощный GPU.

In [ ]:
# Параметры обучения (оптимизированы для мощного GPU)
CONFIG = {
    # Пути
    'data_root': DATASET_PATH,
    'output_dir': f'{BASE_PATH}/working/checkpoints',
    
    # Модель
    'img_size': 256,
    'base_channels': 32,
    'feature_dim': 256,
    'num_attention_heads': 8,
    'dropout': 0.1,
    
    # Обучение
    'batch_size': 16,  # Можно увеличить до 32 на мощном GPU
    'num_epochs': 80,
    'learning_rate': 0.0001,
    'num_workers': 4,
    'device': device,
    
    # Data augmentation
    'augment_level': 'medium',
    
    # Loss
    'regression_weight': 1.0,
    'classification_weight': 0.5,
    'ranking_weight': 0.3,
    'use_smooth_l1': True,
    'margin': 0.5,
    
    # Оптимизация
    'optimizer': 'adamw',
    'weight_decay': 0.0001,
    'scheduler': 'cosine',
    'warmup_epochs': 5,
    'min_lr': 0.000001,
    
    # Training
    'use_amp': True,  # Mixed Precision
    'gradient_accumulation_steps': 1,
    'max_grad_norm': 1.0,
    'early_stopping_patience': 15,
}

# Вывод конфигурации
print("\n⚙️ Конфигурация обучения:")
print("="*60)
for key, value in CONFIG.items():
    print(f"   {key:30s}: {value}")
print("="*60)

## 4️⃣ Инициализация модели и данных

In [ ]:
# Импорт модулей TIGAS
import sys
sys.path.insert(0, BASE_PATH if BASE_PATH != '.' else os.getcwd())

from tigas.models.tigas_model import create_tigas_model
from tigas.data.loaders import create_dataloaders_from_csv
from tigas.training.trainer import TIGASTrainer
from tigas.training.losses import CombinedLoss

print("✅ Модули TIGAS импортированы успешно")

In [ ]:
# Создание модели
print("\n🏗️ Создание модели TIGAS...")
print("="*60)

model = create_tigas_model(
    img_size=CONFIG['img_size'],
    base_channels=CONFIG['base_channels'],
    feature_dim=CONFIG['feature_dim'],
    num_attention_heads=CONFIG['num_attention_heads'],
    dropout=CONFIG['dropout']
)

model = model.to(CONFIG['device'])

model_info = model.get_model_size()
print(f"   ✅ Модель создана:")
print(f"      Параметров: {model_info['total_parameters']:,}")
print(f"      Обучаемых: {model_info['trainable_parameters']:,}")
print(f"      Размер: {model_info['model_size_mb']:.2f} MB")
print(f"      Устройство: {next(model.parameters()).device}")
print("="*60)

In [ ]:
# Создание dataloaders
print("\n📊 Создание dataloaders...")
print("="*60)

dataloaders = create_dataloaders_from_csv(
    data_root=CONFIG['data_root'],
    batch_size=CONFIG['batch_size'],
    img_size=CONFIG['img_size'],
    num_workers=CONFIG['num_workers'],
    augment_level=CONFIG['augment_level'],
    pin_memory=True if CONFIG['device'] == 'cuda' else False
)

print("\n✅ Dataloaders созданы успешно")

In [ ]:
# Создание loss function
loss_fn = CombinedLoss(
    use_tigas_loss=True,
    use_contrastive=False,
    tigas_loss_config={
        'regression_weight': CONFIG['regression_weight'],
        'classification_weight': CONFIG['classification_weight'],
        'ranking_weight': CONFIG['ranking_weight'],
        'use_smooth_l1': CONFIG['use_smooth_l1'],
        'margin': CONFIG['margin']
    }
)

print("✅ Loss function создана")

## 5️⃣ Инициализация тренера

In [ ]:
# Создание тренера
print("\n🎓 Инициализация тренера...")
print("="*60)

os.makedirs(CONFIG['output_dir'], exist_ok=True)

trainer = TIGASTrainer(
    model=model,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    loss_fn=loss_fn,
    optimizer_config={
        'optimizer_type': CONFIG['optimizer'],
        'learning_rate': CONFIG['learning_rate'],
        'weight_decay': CONFIG['weight_decay']
    },
    scheduler_config={
        'scheduler_type': CONFIG['scheduler'],
        'num_epochs': CONFIG['num_epochs'],
        'warmup_epochs': CONFIG['warmup_epochs'],
        'min_lr': CONFIG['min_lr']
    },
    device=CONFIG['device'],
    output_dir=CONFIG['output_dir'],
    use_amp=CONFIG['use_amp'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'],
    max_grad_norm=CONFIG['max_grad_norm'],
    log_interval=10,
    save_interval=5,
    validate_interval=1,
    early_stopping_patience=CONFIG['early_stopping_patience'],
    use_tensorboard=True
)

print("✅ Тренер инициализирован")
print(f"   Чекпоинты будут сохраняться в: {CONFIG['output_dir']}")
print("="*60)

## 6️⃣ Запуск обучения

⏰ **Ожидаемое время обучения:**
- Tesla P100 (16GB): ~6-8 часов для 50-80 эпох
- Tesla T4 (16GB): ~8-12 часов для 50-80 эпох
- Tesla K80 (12GB): ~12-18 часов для 50-80 эпох

💡 **Советы:**
- Следите за прогрессом в реальном времени
- Модель автоматически остановится, если validation loss не улучшается 15 эпох
- Лучшая модель автоматически сохраняется

In [ ]:
# Запуск обучения
print("\n🚀 НАЧАЛО ОБУЧЕНИЯ")
print("="*60)
print(f"   Эпох: {CONFIG['num_epochs']}")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Learning rate: {CONFIG['learning_rate']}")
print(f"   Device: {CONFIG['device']}")
print(f"   Mixed Precision: {CONFIG['use_amp']}")
print("="*60)
print("\n⏰ Приготовьте кофе - это займет несколько часов...\n")

try:
    trainer.train(
        num_epochs=CONFIG['num_epochs'],
        resume_from=None
    )
    
    print("\n" + "="*60)
    print("🎉 ОБУЧЕНИЕ ЗАВЕРШЕНО УСПЕШНО!")
    print("="*60)
    print(f"\n📁 Результаты сохранены в: {CONFIG['output_dir']}")
    print("   - best_model.pt: лучшая модель по validation метрикам")
    print("   - latest_model.pt: последний чекпоинт")
    print("   - logs/: логи TensorBoard")
    
except KeyboardInterrupt:
    print("\n⚠️ Обучение прервано пользователем")
    print(f"   Чекпоинт сохранен в: {CONFIG['output_dir']}/latest_model.pt")
    
except Exception as e:
    print(f"\n❌ Ошибка во время обучения: {e}")
    import traceback
    traceback.print_exc()

## 7️⃣ Визуализация результатов

In [ ]:
# Загрузка TensorBoard в notebook
if IS_COLAB:
    %load_ext tensorboard
    %tensorboard --logdir {CONFIG['output_dir']}/logs
elif IS_KAGGLE:
    print("📊 TensorBoard логи доступны в: {CONFIG['output_dir']}/logs")
    print("   Скачайте логи и просмотрите локально: tensorboard --logdir logs")

In [ ]:
# Проверка финальных метрик
import json
from pathlib import Path

best_model_path = Path(CONFIG['output_dir']) / 'best_model.pt'
if best_model_path.exists():
    checkpoint = torch.load(best_model_path, map_location=CONFIG['device'])
    
    print("\n📊 Финальные метрики лучшей модели:")
    print("="*60)
    if 'metrics' in checkpoint:
        for key, value in checkpoint['metrics'].items():
            print(f"   {key:30s}: {value:.4f}")
    print(f"\n   Эпоха: {checkpoint.get('epoch', 'N/A')}")
    print("="*60)

## 8️⃣ Сохранение и экспорт модели

In [ ]:
# Сжатие результатов для скачивания
import shutil

output_archive = f'{BASE_PATH}/working/tigas_trained_model.zip'

print("📦 Создание архива с результатами...")
shutil.make_archive(
    base_name=output_archive.replace('.zip', ''),
    format='zip',
    root_dir=CONFIG['output_dir']
)

print(f"✅ Архив создан: {output_archive}")
print(f"   Размер: {Path(output_archive).stat().st_size / 1024 / 1024:.1f} MB")

if IS_COLAB:
    from google.colab import files
    print("\n⬇️ Скачивание архива...")
    files.download(output_archive)
elif IS_KAGGLE:
    print("\n💾 Для Kaggle: результаты сохранены в /kaggle/working/")
    print("   Они будут доступны после завершения kernel в Output")

## 🎯 Готово!

Модель обучена и готова к использованию.

### Что дальше?
1. Скачайте `tigas_trained_model.zip`
2. Распакуйте архив
3. Используйте `best_model.pt` для inference

### Как использовать модель?
```python
# Загрузка модели
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Inference
with torch.no_grad():
    output = model(image)
    prediction = output['score']  # 0.0 = fake, 1.0 = real
```

---

**Спасибо за использование TIGAS! 🚀**